## Oi, amigos

Como é só revisar essa parte, estou colocando assim:

---
Markdown original
#### Beware of David's diet!
---
Markdown traduzido
#### Se liga na dieta do David!
---

Então, vocês vão revisando, apagando o texto original e deixando apenas os markdowns revisados 

De vez em quando vão aparecer uns quatro asteríscos: eu não tive uma ideia boa para traduzir naquele momento e continuei com a tradução

Vamos a o que interessa:

## Glossário

Apenas algumas traduções que pesquisei ou adotei e que achei boas o suficiente pra repetir ela em toda ocorrência do termo

- To Provide : Fornece
- Chart : Gráfico
- To Highlight : Destacar
- To Link : Interligar
- Interaction : Interação
- Selection : Seleção
- Mark : Marca
- Encoding : Codificação
- Notebook : Notebook
- Dataset : Dataset
- Scatter Plot : Gráfico de Dispersão
- Mouse (de pc) : Mouse
- Single : Único   (como em single value selections)
- Widget : Widget
- Data Fields : Campo de Dados
- Enable : Habilitar
- Drop-Down Menu : Menu Drop-Down


# Interaction

_“A graphic is not ‘drawn’ once and for all; it is ‘constructed’ and reconstructed until it reveals all the relationships constituted by the interplay of the data. The best graphic operations are those carried out by the decision-maker themself.”_ &mdash; [Jacques Bertin](https://books.google.com/books?id=csqX_xnm4tcC)

Visualization provides a powerful means of making sense of data. A single image, however, typically provides answers to, at best, a handful of questions. Through _interaction_ we can transform static images into tools for exploration: highlighting points of interest, zooming in to reveal finer-grained patterns, and linking across multiple views to reason about multi-dimensional relationships.

At the core of interaction is the notion of a _selection_: a means of indicating to the computer which elements or regions we are interested in. For example, we might hover the mouse over a point, click multiple marks, or draw a bounding box around a region to highlight subsets of the data for further scrutiny.

Alongside visual encodings and data transformations, Altair provides a _selection_ abstraction for authoring interactions. These selections encompass three aspects:

1. Input event handling to select points or regions of interest, such as mouse hover, click, drag, scroll, and touch events.
2. Generalizing from the input to form a selection rule (or [_predicate_](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29)) that determines whether or not a given data record lies within the selection.
3. Using the selection predicate to dynamically configure a visualization by driving _conditional encodings_, _filter transforms_, or _scale domains_.

This notebook introduces interactive selections and explores how to use them to author a variety of interaction techniques, such as dynamic queries, panning &amp; zooming, details-on-demand, and brushing &amp; linking.

_This notebook is part of the [data visualization curriculum](https://github.com/uwdata/visualization-curriculum)._

---

# Interação

_"Um gráfico não é 'desenhado' de uma vez por todas; é 'construído' e reconstruído até que revele todas as relações constituída pela interação dos dados. As melhores operações gráficas são aquelas implementadas pelo próprio tomador de decisão."_ &mdash; [Jacques Bertin](https://books.google.com/books?id=csqX_xnm4tcC)

Visualização fornece meios poderosos para making sense **** dos dados. Uma única imagem, porém, tipicamente fornece respostas a, no máximo, algumas poucas perguntas. Através da _interaction_ podemos transformar imagens estáticas em ferramentas para exploração: destacando pontos de interessee, dando zoom para revelar padrões finos e interligando através de várias visualizações para entender sobre relações multidimensionais.

No cerne na interação está a noção de uma _seleção_: um meio de indicar para o computador quais elementos ou regiões estamos interessados em. Por exemplo, nós podemos pairar o mouse sobre um ponto, clicar em várias marcas,  ou desenhar uma moldura em volta de uma região para destacar um pedaço dos dados para análise mais aprofundada.

Junto às codificações visuais e às transformações de dados, o Altair fornece uma abstração de _seleção_ para a criação de interações. Essas seleções englobam três aspectos:

1. Lidar com eventos de entrada para selecionar pontos ou regiões de interesse, como o pairar do mouse, eventos de clicar, de arrastar, de rolar, e de tocar.
2. Generalizar a partir da entrada para construir uma regra de seleção (ou [_predicado_](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29)) que determinar se um registro de um dado fornecido está na seleção ou não.
3. Usar o predicado de seleção para configurar dinamicamente uma visualização através de _codificações condicionais_, _transformações de filtro_ ou _domínios de escala_.

Esse notebook introduz seleções interativas e explora como usá-las para criar uma variedade de técnicas de interação, como consultas dinâmicas, panorama &amp; zoom, detalhes em demanda, e interligar &amp; varrer.  

_Esse notebook é parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._

In [ ]:
import pandas as pd
import altair as alt

## Datasets

We will visualize a variety of datasets from the [vega-datasets](https://github.com/vega/vega-datasets) collection:

- A dataset of `cars` from the 1970s and early 1980s,
- A dataset of `movies`, previously used in the [Data Transformation](https://github.com/uwdata/visualization-curriculum/blob/master/altair_data_transformation.ipynb) notebook,
- A dataset containing ten years of [S&amp;P 500](https://en.wikipedia.org/wiki/S%26P_500_Index) (`sp500`) stock prices,
- A dataset of technology company `stocks`, and
- A dataset of `flights`, including departure time, distance, and arrival delay.
---

## Datasets

Nós vamos visualizar uma variedade de datasets da coleção [vega-datasets](https://github.com/vega/vega-datasets):

- Um dataset `cars` de carros dos anos 70 e começo dos anos 80,
- Um dataset `movies` de filmes, anteriormente usados no notebook [Transformação de Dados](https://github.com/uwdata/visualization-curriculum/blob/master/altair_data_transformation.ipynb),
- Um dataset contendo dez anos de [S&amp;P 500](https://en.wikipedia.org/wiki/S%26P_500_Index) (`sp500`) preços de ações,
- Um dataset `stocks` das ações de empresas de tecnologia e
- Um dataset `flights` dos voos de avião, incluindo hora de decolagem, distância e atraso de pouso

In [ ]:
cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

## Introducing Selections

Let's start with a basic selection: simply clicking a point to highlight it. Using the `cars` dataset, we'll start with a scatter plot of horsepower versus miles per gallon, with a color encoding for the number cylinders in the car engine.

In addition, we'll create a selection instance by calling `alt.selection_single()`, indicating we want a selection defined over a _single value_. By default, the selection uses a mouse click to determine the selected value. To register a selection with a chart, we must add it using the `.add_selection()` method.

Once our selection has been defined, we can use it as a parameter for _conditional encodings_, which apply a different encoding depending on whether a data record lies in or out of the selection. For example, consider the following code:

~~~ python
color=alt.condition(selection, 'Cylinders:O', alt.value('grey'))
~~~

This encoding definition states that data points contained within the `selection` should be colored according to the `Cylinder` field, while non-selected data points should use a default `grey`. An empty selection includes _all_ data points, and so initially all points will be colored.

_Try clicking different points in the chart below. What happens? (Click the background to clear the selection state and return to an "empty" selection.)_

---

## Introduzindo Seleções

Vamos começar com uma seleção básica: simplesmente clicando num ponto para destacá-lo. Usando o dataset `cars`, vamos começão com um gráfico de dispersão da potência (em cavalos) por milhas por galão, com uma codificação de cor para o número de cilindros no motor do carro.

Além disso, vamos criar uma instância de seleção chamando `alt.selection_single()`, indicando que queremos uma seleção definida por um _único valor_. Por padrão, a seleção usa um clique de mouse para determinar o valor selecionado. Para registrar uma seleção com um gráfico, devemos adicionar usando o método `.add_selection()`.

Uma vez que nossa seleção tinha sido definida, podemos usá-la como parâmetro para _codificações condicionais_, que aplicam uma codificação diferente dependendo se o registro do dado está dentro ou fora da seleção. Por exemplo, considere o seguinte código:

~~~ python
color=alt.condition(selection, 'Cylinders:O', alt.value('grey'))
~~~

Essa definição de codificação afirma que os pontos do dado contidos em `selection` devem ser coloridos de acordo com o campo `Cylinder`, enquanto pontos do dado não selecionados devem usar um `grey` (_cinza_) padrão. Uma seleção vazia inclui _todos_ os pontos do dado e, então, inicialmente todos os pontos serão coloridos.

_Tente clicar em diferentes pontos no gráfico abaixo. O que acontece? (Clique no fundo para limpar o estado da seleção e retornar a uma seleção "vazia".)_


In [ ]:
selection = alt.selection_single();
  
alt.Chart(cars).mark_circle().add_selection(
    selection
).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
)

Of course, highlighting individual data points one-at-a-time is not particularly exciting! As we'll see, however, single value selections provide a useful building block for more powerful interactions. Moreover, single value selections are just one of the three selection types provided by Altair:

- `selection_single` - select a single discrete value, by default on click events. 
- `selection_multi` - select multiple discrete values. The first value is selected on mouse click and additional values toggled using shift-click. 
- `selection_interval` - select a continuous range of values, initiated by mouse drag.

Let's compare each of these selection types side-by-side. To keep our code tidy we'll first define a function (`plot`) that generates a scatter plot specification just like the one above. We can pass a selection to the `plot` function to have it applied to the chart:

---

Claro, destacar pontos do dado individuais um a um não é particularmente divertido! Como veremos, seleções de valor único fornecem um alicerce útil para construir interações mais poderosas. Ademais, seleções de valor único são apenas uma dos três tipos de seleção fornecidas pelo Altair:

- `selection_sigle` - selecione um único valor discreto, por padrão em eventos de clique.
- `selection_multi` - selecione vários valores discretos. O primeiro valor é selecionado clicando com o mouse e valores adicionais são inseridos usando Shift+Click. 
- `selection_interval` - selecione uma faixa contínua de valores, iniciados arrastando o mouse enquanto clicado.

Vamos comparar cada uma desses tipos de seleção lado a lado. Para manter nosso código arrumado, vamos primeiro definir uma função (`plot`) que gera especificações dos gráficos de dispersão como o acima. Podemos passar uma seleção para a função `plot` para aplicá-lo ao gráfico:

In [ ]:
def plot(selection):
    return alt.Chart(cars).mark_circle().add_selection(
        selection
    ).encode(
        x='Horsepower:Q',
        y='Miles_per_Gallon:Q',
        color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
    ).properties(
        width=240,
        height=180
    )

Let's use our `plot` function to create three chart variants, one per selection type.

The first (`single`) chart replicates our earlier example. The second (`multi`) chart supports shift-click interactions to toggle inclusion of multiple points within the selection. The third (`interval`) chart generates a selection region (or _brush_) upon mouse drag. Once created, you can drag the brush around to select different points, or scroll when the cursor is inside the brush to scale (zoom) the brush size.

_Try interacting with each of the charts below!_

---

Vamos usar nossa função `plot` para criar três variantes do gráfico, um para cada tipo de seleção.

O primeiro gráfico (`single`) replica nosso exemplo anterior. O segundo gráfico (`multi`) suporta a interação Shift+Click para inserir vários pontos na seleção. O terceiro gráfico (`interval`) gera uma região selecionada (ou _varrida_) a partir do arrasto do mouse clicado. Uma vez criada, você pode mover a área varrida pelo gráfico para selecionar diferentes pontos, ou rolar quando o cursor está dentro da área varrida para aumentar ou diminuir o tamanho da área.

_Tente interagir com os gráficos abaixo!_

In [ ]:
alt.hconcat(
  plot(alt.selection_single()).properties(title='Single (Click)'),
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),
  plot(alt.selection_interval()).properties(title='Interval (Drag)')
)

The examples above use default interactions (click, shift-click, drag) for each selection type. We can further customize the interactions by providing input event specifications using [Vega event selector syntax](https://vega.github.io/vega/docs/event-streams/). For example, we can modify our `single` and `multi` charts to trigger upon `mouseover` events instead of `click` events.

_Hold down the shift key in the second chart to "paint" with data!_

---

Os exemplos acima usam interações padrão (Clicar, Shift+Click, Arrastar) para cada tipo de seleção. Podemos costumizar mais as interações fornecendo especificações do evento de entrada usando a [sintaxe de seleção de evento do Vega](https://vega.github.io/vega/docs/event-streams/). Por exemplo, podemos modificar nossos gráficos `single` e `multi` para acionar com eventos de `pairar do mouse` ao invés de eventos de `click`

_Segure a tecla Shift no segundo gráfico para "pintar" com os dados!_

In [ ]:
alt.hconcat(
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Mouseover)'),
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Mouseover)')
)

Now that we've covered the basics of Altair selections, let's take a tour through the various interaction techniques they enable!

---

Agora que cobrimos o básico de seleções do Altair, vamos fazer um tour pelas várias técnicas de interação que eles habilitam! 

## Dynamic Queries

_Dynamic queries_ enables rapid, reversible exploration of data to isolate patterns of interest. As defined by [Ahlberg, Williamson, &amp; Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf), a dynamic query:

- represents a query graphically,
- provides visible limits on the query range,
- provides a graphical representation of the data and query result,
- gives immediate feedback of the result after every query adjustment,
- and allows novice users to begin working with little training.

A common approach is to manipulate query parameters using standard user interface widgets such as sliders, radio buttons, and drop-down menus. To generate dynamic query widgets, we can apply a selection's `bind` operation to one or more data fields we wish to query.

Let's build an interactive scatter plot that uses a dynamic query to filter the display. Given a scatter plot of movie ratings (from Rotten Tomates and IMDB), we can add a selection over the `Major_Genre` field to enable interactive filtering by film genre.

---

## Consultas Dinâmicas

_Consultas dinâmicas_ habilitam um exploração reversível dos dados rápida para isolar padrões de interesse. Como definido por [Ahlberg, Williamson, &amp; Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf), uma consulta dinâmica:

- representa uma consulta graficamente,
- fornece limites visuais para a faixa da consulta,
- fornece uma representação gráfica dos dados e o resultado da consulta,
- dá um retorno imediato do resultado após cada ajuste da consulta,
- e possibilita usuários novatos começar a trabalhar com pouco treino.

Uma abordagem comum é manipular os parâmetros da consulta usando ferramentas da interface do usuário padrão como uma barra deslizante, botões de rádio, e menus drop-down. Para gerar ferramentas de consulta dinâmica, podemos aplicar a operação `bind` da seleção para um ou mais campos de dados que queremos consultar.

Vamos construir um gráfico de dispersão interativo que usa uma consulta dinâmica para filtrar a exibição. Dado um gráfico de dispersão de avaliações de filmes (do Rotten Tomatoes e IMDB), podemos adicionar uma seleção sobre o campo `Major_Genre` para habilitar filtragens por gênero interativas.




To start, let's extract the unique (non-null) genres from the `movies` data:

---

Para começar, vamos extrair os gêneros (não nulos) dos dataset `movies`.

In [ ]:
df = pd.read_json(movies) # load movies data
genres = df['Major_Genre'].unique() # get unique field values
genres = list(filter(lambda d: d is not None, genres)) # filter out None values
genres.sort() # sort alphabetically

For later use, let's also define a list of unique `MPAA_Rating` values:

---

Para uso futuro, vamos definir também uma lista de valores `MPAA_Rating` sem repetição:

In [ ]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Now let's create a `single` selection bound to a drop-down menu.

*Use the dynamic query menu below to explore the data. How do ratings vary by genre? How would you revise the code to filter `MPAA_Rating` (G, PG, PG-13, etc.) instead of `Major_Genre`?*

---

Agora vamos criar uma `única` seleção ligada a um menu drop-down.

*Use o menu de consulta dinâmica abaixo para explorar os dados. Como as avaliações variam por gênero? Como você revisaria o código para filtar `MPAA_Rating` (G, PG, PG-13, etc.) ao invés de `Major_Genre`?*

In [ ]:
selectGenre = alt.selection_single(
    name='Select', # name the selection 'Select'
    fields=['Major_Genre'], # limit selection to the Major_Genre field
    init={'Major_Genre': genres[0]}, # use first genre entry as initial value
    bind=alt.binding_select(options=genres) # bind to a menu of unique genre values
)

alt.Chart(movies).mark_circle().add_selection(
    selectGenre
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))
)

Our construction above leverages multiple aspects of selections:

- We give the selection a name (`'Select'`). This name is not required, but allows us to influence the label text of the generated dynamic query menu. (_What happens if you remove the name? Try it!_)
- We constrain the selection to a specific data field (`Major_Genre`). Earlier when we used a `single` selection, the selection mapped to individual data points. By limiting the selection to a specific field, we can select _all_ data points whose `Major_Genre` field value matches the single selected value.
- We initialize `init=...` the selection to a starting value.
- We `bind` the selection to an interface widget, in this case a drop-down menu via `binding_select`.
- As before, we then use a conditional encoding to control the opacity channel.

---

Nossa construção acima